In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-02-17 23:27:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-02-17 23:27:56 (17.9 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
len(text)

1115394

In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



All unique chars in text

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Tokenize characters

In [7]:
text_int = {ch:i for i, ch in enumerate(chars)}
int_text = {i:ch for i, ch in enumerate(chars)}

encode = lambda x: [text_int[i] for i in x]
decode  = lambda x: ''.join([int_text[i] for i in x])

In [8]:
encode('hi there')

[46, 47, 1, 58, 46, 43, 56, 43]

In [9]:
decode(encode('hi there'))

'hi there'

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)

Split Data

In [11]:
n = int(0.9*len(text))
train_data = data[:n]
valid_data = data[n:]

In [12]:
block_size = 8
data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = data[:block_size]
y = data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'context is {context} and target is {target}')

context is tensor([18]) and target is 47
context is tensor([18, 47]) and target is 56
context is tensor([18, 47, 56]) and target is 57
context is tensor([18, 47, 56, 57]) and target is 58
context is tensor([18, 47, 56, 57, 58]) and target is 1
context is tensor([18, 47, 56, 57, 58,  1]) and target is 15
context is tensor([18, 47, 56, 57, 58,  1, 15]) and target is 47
context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target is 58


In [14]:
torch.manual_seed(43)
batch_size=4 #how many independent seq to process in parallel
block_size=8 #sets the size of the context

def get_batch(split):
    '''
        Generate a small batch of data of input x and target y
    '''

    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb.shape)
print(yb.shape)



torch.Size([4, 8])
torch.Size([4, 8])


In [15]:
xb

tensor([[43,  1, 58, 53,  1, 51, 43,  6],
        [ 1, 57, 46, 43,  1, 61, 39, 57],
        [39, 42, 43,  1, 46, 47, 51,  1],
        [53, 59, 56,  1, 45, 56, 39, 41]])

In [16]:
yb

tensor([[ 1, 58, 53,  1, 51, 43,  6,  1],
        [57, 46, 43,  1, 61, 39, 57,  1],
        [42, 43,  1, 46, 47, 51,  1, 58],
        [59, 56,  1, 45, 56, 39, 41, 43]])

In [17]:
for b in range(batch_size): #block size
    for c in range(block_size):
        context = xb[b, :c+1]
        target = yb[b, c]
        print(f"Context:: {context}, Target:: {target}")



Context:: tensor([43]), Target:: 1
Context:: tensor([43,  1]), Target:: 58
Context:: tensor([43,  1, 58]), Target:: 53
Context:: tensor([43,  1, 58, 53]), Target:: 1
Context:: tensor([43,  1, 58, 53,  1]), Target:: 51
Context:: tensor([43,  1, 58, 53,  1, 51]), Target:: 43
Context:: tensor([43,  1, 58, 53,  1, 51, 43]), Target:: 6
Context:: tensor([43,  1, 58, 53,  1, 51, 43,  6]), Target:: 1
Context:: tensor([1]), Target:: 57
Context:: tensor([ 1, 57]), Target:: 46
Context:: tensor([ 1, 57, 46]), Target:: 43
Context:: tensor([ 1, 57, 46, 43]), Target:: 1
Context:: tensor([ 1, 57, 46, 43,  1]), Target:: 61
Context:: tensor([ 1, 57, 46, 43,  1, 61]), Target:: 39
Context:: tensor([ 1, 57, 46, 43,  1, 61, 39]), Target:: 57
Context:: tensor([ 1, 57, 46, 43,  1, 61, 39, 57]), Target:: 1
Context:: tensor([39]), Target:: 42
Context:: tensor([39, 42]), Target:: 43
Context:: tensor([39, 42, 43]), Target:: 1
Context:: tensor([39, 42, 43,  1]), Target:: 46
Context:: tensor([39, 42, 43,  1, 46]), 

BiGram Language Model

In [26]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        #each token directly reads the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        #idx and targets are both (Batch ,Time) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch=batch_size, Time=block_size, Channel=vocab_size)

        if targets is not None:
            B, T, C = logits.shape

            #For pytorch loss module, reshaping
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is B,T array in current context
        for _ in range(max_new_tokens):
            #get the preds
            logits, loss = self(idx)
            #get the last time step
            logits = logits[:, -1, :] # B,C
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # B,C
            #sample from dist
            idx_next = torch.multinomial(probs, num_samples=1) #B,1
            #append
            idx = torch.cat((idx, idx_next), dim=1) #B,T+1
        return idx



m = BigramLanguageModel(vocab_size)
out, loss = m(xb,yb)
print(out.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.5094, grad_fn=<NllLossBackward0>)


OMGBk.Mhel CwYya!'GlReTTm!&GNJIXBPvxL.E
ZzoeH?FBlNjdJ
O!cXJjyjgLus$'?3SQ.ERDjUPfVvxM&Q:iwmYTVvGiMDE


Train the Model

In [27]:
#Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [37]:
batch_size=32
for epoch in range(1000):

    #sample a batch of data 
    xb, yb = get_batch('train')

    #eval loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.521436929702759


In [41]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


COR: ssfe;
EOLurulat teisVIEYot!
T:

MAnderny hilllaus moveeTin hedis he yo w&CAUSTI tho c
By toug.
hthinortad theis m whares wom&Qerat tath ss Pr I lboobe ist pBRishy cor air prior se te nomeaineand ou hat our d s y blsksocimerethervere sincor'stegeas Ither tre!
I'storitheand! tor obe akeco icave poqu hee isthand prd le ma.
Thethalless!
Th IVpe whoresecuf f tal po t ie o thaveetaulRI'd bendedur GEDWherP.
IVim sthyoupat, Moupe r
Pr?
Bll howithiby, hey be t
Fathate'se 'They we emendangg bbed thel


### Self Attention Mathematical trick

In [42]:
B, T, C = 4, 8, 2 #Batch, Time, Channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

Provide info to the current token with all the past tokens: i.e average

In [43]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [44]:
x[0]

tensor([[-0.8987, -0.0352],
        [-1.5111, -2.0649],
        [ 0.7211,  1.6755],
        [ 0.8848, -0.9132],
        [ 0.3272, -1.1490],
        [ 1.4924,  0.4495],
        [ 1.3127, -1.4060],
        [-1.4175,  0.7419]])

In [45]:
xbow[0]

tensor([[-0.8987, -0.0352],
        [-1.2049, -1.0500],
        [-0.5629, -0.1415],
        [-0.2010, -0.3344],
        [-0.0953, -0.4974],
        [ 0.1693, -0.3395],
        [ 0.3326, -0.4919],
        [ 0.1139, -0.3377]])